In [16]:
import os
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnableParallel
from langchain_core.prompts import format_document
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
MODEL = "digilawyer1.4"


In [17]:
from langchain_community.llms import Ollama

model = Ollama(model=MODEL, temperature=0)

In [3]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Search(BaseModel):
    """Search over a database of job records."""

    queries: List[str] = Field(
        ...,
        description="Distinct queries to search for",
    )

In [29]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

output_parser = PydanticToolsParser(tools=[Search])

system = """You have the ability to issue search queries to get information to help answer user information.

Generate the similar queries of the given query

Query. [Write a question here]
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
# structured_llm =   (Search)
query_analyzer = {"question": RunnablePassthrough()} | prompt | model

In [30]:
query_analyzer.invoke("I had an car accident on highway.")

'\nSimilar Queries:\n\n1. What should I do after a car accident?\n2. How to file a car accident claim?\n3. Can I get compensation for my car accident injuries?\n4. How to prove fault in a car accident?\n5. What are the common causes of car accidents?'

In [31]:
query_analyzer.invoke("My company is not giving my salary for 3 months. Can you tell me BNSS legal rights I have for this?")

"\nSure, as a DigiLawyer, I can help you with that! Based on the context and question provided, here are some potential queries that could provide more information on your legal rights:\n\n1. Can you please provide more details about your employment contract or any relevant company policies regarding salary payments?\n2. Have you tried contacting your HR department or supervisor to inquire about the delay in salary payment?\n3. Are there any specific provisions in Indian labor laws that may apply to your situation, such as the Payment of Gratuity Act or the Employees' State Insurance Act?\n4. Have you considered seeking legal advice from a qualified attorney specializing in employment law?\n5. Are there any alternative dispute resolution mechanisms available to resolve the issue, such as mediation or arbitration?"

In [13]:
model.get_input_schema()

pydantic.v1.main.OllamaInput